In [76]:
import xml.etree.ElementTree as ET
from sklearn import feature_extraction
import sklearn as skl
from nltk.stem import RSLPStemmer
import nltk
from zipfile import ZipFile
import pandas as pd
import numpy as np
import unicodedata
import os
import re
import gensim
import scipy.sparse
from gensim.models import Word2Vec
from sklearn.metrics import roc_auc_score, make_scorer
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForPreTraining
from transformers import AutoModel
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import sklearn.preprocessing as skl_preprocessing
import sklearn.metrics as skl_metrics
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, make_scorer
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForPreTraining
from transformers import AutoModel

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/af_ia/

!ls

/content/drive/MyDrive/af_ia
db  files


In [4]:
# Caminho dos arquivos extraidos do kaggle
path_dataset = r'files/classificao-de-notcias.zip'
path_db = r'db'

# Caminho dos arquivos que serão utilizados para a atividade
path_train = r'db/arquivos_competicao/arquivos_competicao/train.csv'
path_test = r'db/arquivos_competicao/arquivos_competicao/test.csv'
path_news = r'db/arquivos_competicao/arquivos_competicao/news'


In [5]:
def unzip(path, pathFolder):

    # descompacta a base de dados de notícias
    z = ZipFile(path, 'r')

    if os.path.isdir(pathFolder):
        z.extractall(pathFolder)
        z.close()
    else:
        os.mkdir(pathFolder)
        z.extractall(pathFolder)
        z.close()

    print("Arquivo descompactado com sucesso!")

# Antes de descompactar os arquivos valida se ja foram descompactados antes
if not os.path.isdir(path_news):
    unzip(path_dataset, path_db)
else:
    print("Arquivo já descompactado")

Arquivo já descompactado


---
## 1. Carregando os arquivos de teste e treino

In [6]:
#Carregando os arquivos de treino
df_train = pd.read_csv(path_train)
print("Colunas do arquivo de treino: ", df_train.columns)
print("Quantidade de linhas: ", df_train.shape[0])

Colunas do arquivo de treino:  Index(['ID', 'Class'], dtype='object')
Quantidade de linhas:  2781


In [7]:
# Carregando os arquivos de teste
df_teste = pd.read_csv(path_test)
print("Colunas do arquivo de treino: ", df_teste.columns)
print("Quantidade de linhas: ", df_teste.shape[0])

Colunas do arquivo de treino:  Index(['ID'], dtype='object')
Quantidade de linhas:  1193


---
## Pré-Processamento dos Dados


### Criando um DataFrame com os textos e titulos extraidos do XML
Para facilitar a aplicação dos metodos foi adicionando novas colunas no DataFrame de treino e teste, contendo o texto e titulo extraidos dos arquivos xml

In [8]:
def extract_xml_text(path_xml):

    """
    A função `extract_xml_text` é designada para extrair os textos dos arquivos XML
    especificados pelo `path_xml`.
    Utilizando a biblioteca ElementTree para leitura do arquivo XML
    """

    # Instancia um objeto como uma árvore de análise
    tree = ET.parse(path_xml)

    # Obtem o elemento raiz da árvore de ánalise
    root = tree.getroot()

    # Encontra o elemento headline (titulo) dentro da árvore de analise
    headline = root.find('headline').text if root.find('headline') is not None else ''

    # Entroa todos os elementos <p> que na estrutura dos xml's contem o texto
    paragraphs = root.findall('.//p')

    # Junta em uma unica string, separando por espaços
    text = ' '.join([p.text for p in paragraphs if p.text is not None])

    return headline, text

def apply_extraction(df_applyed):

    """
    Essa função é responsável em aplicar as novas colunas
    no df_applyed passado como parametro.
    """

    # Loop pelas linhas do df
    for idx in df_applyed.index:

        # atribui o valor da coluna id na variavel file
        file = df_applyed.at[idx, 'ID']

        # Concatena o nome do arquivo com o caminho dele
        path_xml = f"{path_news}/{file}"

        # Extrai o texto e titulo desse arquivo
        titulo, texto = extract_xml_text(path_xml)

        #Atribui esses o texto e titulos em novas colunas
        df_applyed.at[idx, 'TITULO'] = titulo
        df_applyed.at[idx, 'TEXTO'] = texto

    return df_applyed

def print_porcentagem(target):
    # Calcula a contagem de cada classe
    class_counts = target['Class'].value_counts()

    # Calcula a porcentagem de cada classe
    class_percentages = class_counts / len(target) * 100

    # Imprime a porcentagem de cada classe
    for cl, pct in class_percentages.items():
        print(f"Porcentagem da classe {cl}: {round(pct, 2)}%")

def preprocessing_portuguese(text, stemming = False, stopwords = False):
    """
    Funcao usada para tratar textos escritos na lingua portuguesa

    Parametros:
        text: variavel do tipo string que contem o texto que devera ser tratado

        stemming: variavel do tipo boolean que indica se a estemizacao deve ser aplicada ou nao

        stopwords: variavel do tipo boolean que indica se as stopwords devem ser removidas ou nao
    """

    # Lower case
    text = text.lower()

    # remove os acentos das palavras
    nfkd_form = unicodedata.normalize('NFKD', text)
    text = u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

    # remove tags HTML
    regex = re.compile('<[^<>]+>')
    text = re.sub(regex, " ", text)

    # normaliza as URLs
    regex = re.compile('(http|https)://[^\s]*')
    text = re.sub(regex, "<URL>", text)

    # normaliza emails
    regex = re.compile('[^\s]+@[^\s]+')
    text = re.sub(regex, "<EMAIL>", text)

    # converte todos os caracteres não-alfanuméricos em espaço
    regex = re.compile('[^A-Za-z0-9]+')
    text = re.sub(regex, " ", text)

    # normaliza os numeros
    regex = re.compile('[0-9]+.[0-9]+')
    text = re.sub(regex, "NUMERO", text)

    # normaliza os numeros
    regex = re.compile('[0-9]+,[0-9]+')
    text = re.sub(regex, "NUMERO", text)

    # normaliza os numeros
    regex = re.compile('[0-9]+')
    text = re.sub(regex, "NUMERO", text)


    # substitui varios espaçamentos seguidos em um só
    text = ' '.join(text.split())

    # separa o texto em palavras
    words = text.split()

    # trunca o texto para apenas 200 termos
    words = words[0:200]

    # remove stopwords
    if stopwords:
        words = text.split() # separa o texto em palavras
        words = [w for w in words if not w in nltk.corpus.stopwords.words('portuguese')]
        text = " ".join( words )

    # aplica estemização
    if stemming:
        stemmer_method = RSLPStemmer()
        words = [ stemmer_method.stem(w) for w in words ]
        text = " ".join( words )

    # remove palavras compostas por apenas um caracter
    words = text.split() # separa o texto em palavras
    words = [ w for w in words if len(w)>1 ]
    text = " ".join( words )

    return text

In [9]:
df_train = apply_extraction(df_train)
df_train = df_train[['ID', 'TITULO', 'TEXTO', 'Class']]
print_porcentagem(df_train)

Porcentagem da classe Mercados: 74.33%
Porcentagem da classe Economia: 21.18%
Porcentagem da classe GovSocial: 3.24%
Porcentagem da classe CorpIndustrial: 1.26%


In [111]:
# Transformação do target
le = LabelEncoder()
df_train['Class'] = le.fit_transform(df_train['Class'])

In [10]:
df_teste = apply_extraction(df_teste)
df_teste = df_teste[['ID', 'TITULO', 'TEXTO']]

### Tratando os textos da base de dados
- Aplicada a função de estemização para a linguagem dos textos (português)
- Removendo os ascentos das palavras
- Criando um limite de 200 temrmos por palavras, para evitar que a predição do classificador seja influenciada pelo tamanho da noticia.

In [11]:
# Download the stopwords corpus
nltk.download('stopwords')

# Download the RSLPStemmer
nltk.download('rslp')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [12]:
# Aplicar a função ao DataFrame de treino
df_train['TEXTO'] = df_train['TEXTO'].apply(preprocessing_portuguese)

# Aplicando a função no df de teste
df_teste['TEXTO'] = df_teste['TEXTO'].apply(preprocessing_portuguese)

### Fazendo a divisão entre os dados de teste e treino

In [13]:
# gera uma divisão dos dados em treino e teste
cv = skl.model_selection.StratifiedShuffleSplit(n_splits=1, train_size=0.8,
                                                random_state=10)

# retorna os índices de treino e teste
dataset = df_train['TEXTO']
target = df_train['Class']
train_index, test_index = list( cv.split(dataset, target) )[0]

# retorna as partições de treino e teste de acordo com os índices
dataset_train, dataset_test = dataset[train_index], dataset[test_index]
Y_train, Y_test = target[train_index], target[test_index]

print('Qtd. dados de treinamento: %d (%1.2f%%)' %(dataset_train.shape[0], (dataset_train.shape[0]/dataset.shape[0])*100) )
print('Qtd. de dados de teste: %d (%1.2f%%)' %(dataset_test.shape[0], (dataset_test.shape[0]/dataset.shape[0])*100) )

Qtd. dados de treinamento: 2224 (79.97%)
Qtd. de dados de teste: 557 (20.03%)


### Gerando a representação vetorial

#### Gerando a representação vetorial para TF

Iremos transformar o texto em um vetor de atributos com valores numéricos. Uma das formas de fazer isso é considerar que cada palavra (ou token) da base de dados de treinamento é um atributo que armazena o número de vezes que uma determinada palavra aparece no texto. Na biblioteca `scikit-learn` podemos fazer essa conversão de texto para um vetor de atributos usando a função `skl.feature_extraction.text.CountVectorizer()`. Essa função gera um modelo de vetorização que pode ser ajustado com a base nos dados de treinamento usando a função `fit_transform()`.

Obs.: você deve treinar o modelo de representação **apenas com os dados de treinamento**.

In [84]:
# inicializa o modelo usado para gerar a representação TF (term frequency)
vectorizer = skl.feature_extraction.text.CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                                                         stop_words = None, lowercase = True, binary=False, dtype=np.int32)

# treina o modelo TF com os dados de treinamento e converte os dados de treinamento para uma array que contém a frequência dos termos em cada documento (TF - term frequency)
X_train_tf = vectorizer.fit_transform(dataset_train)

# converte os dados de teste
X_test_tf = vectorizer.transform(dataset_test)

print('20 primeiras palavras do vocabulário obtidas a partir dos dados de treinamento:\n')
print(vectorizer.get_feature_names_out()[0:20])

print('\nDimensão dos dados vetorizados: ', X_train_tf.shape)
print('\nDimensão dos dados vetorizados: ', X_test_tf.shape)

20 primeiras palavras do vocabulário obtidas a partir dos dados de treinamento:

['aa' 'aaa' 'aanumero' 'aas' 'abaixo' 'abaixos' 'abaixou' 'abalada'
 'abanadas' 'abanar' 'abandona' 'abandonado' 'abandonaram' 'abastecimento'
 'abatidas' 'abatimentos' 'aberta' 'abertamente' 'abertas' 'aberto']

Dimensão dos dados vetorizados:  (2224, 9506)

Dimensão dos dados vetorizados:  (557, 9506)


#### Gerando a representação vetorial para binário

Utilizando a técnica de vetorização binária onde cada termo é representado como um valor binário. Ou seja, se o termo aparece no documento o valor é 1, caso contrário o valor é 0

In [85]:
X_train_bin = X_train_tf.copy()
X_test_bin = X_test_tf.copy()

#convert os dados de treino para representação binária
X_train_bin[X_train_bin!=0]=1

#convert os dados de teste para representação binária
X_test_bin[X_test_bin!=0]=1

print(X_train_bin.shape)
print(X_test_bin.shape)

(2224, 9506)
(557, 9506)


#### TF-IDF

Utilizando a função **TfidfTransformer** para para converter a representação vetorial TF para TF-IDF
Os parametros utilizados nessa função foram:
- **norm=l2**: normalizando cada vetor TF-IDF para que a soma dos quadrados dos elementos seja igual a 1
- **smooth_idf**: adiciona 1 ao denominador da formula de IDF para eviter divisoes por zero
- **sublinear_tf**: aplica a sublinearização do TF

In [86]:
#Inicializa o modelo usado para gerar a representação TF-IDF
tfidf_model = skl.feature_extraction.text.TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

# Treina o modelo com os vetores de treinamento
X_train_tfidf = tfidf_model.fit_transform(X_train_tf)

# treina o modelo com os dados de teste
X_test_tfidf = tfidf_model.transform(X_test_tf)

print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(2224, 9506)
(557, 9506)


### Gerando word embeddings


#### Gerando com a própria base

Depois de fazer o pré-processamento, é necessário transformar o texto em um vetor de atributos com valores numéricos. Podemos fazer isso usando word embeddings pré-treinadas ou treinando um modelo próprio.

Vamos passar por toda a base de dados e transformar cada documento em uma lista de palavra.

In [87]:
dataset2_train = []
for i, msg in enumerate(dataset_train):
    dataset2_train.append(msg.split())

dataset2_test = []
for i, msg in enumerate(dataset_test):
    dataset2_test.append(msg.split())

print("\n\n20 primeiras palavras da primeira amostra de treino")
print(dataset2_train[0][0:30])

print("\n\n20 primeiras palavras da primeira amostra de teste")
print(dataset2_test[0][0:30])



20 primeiras palavras da primeira amostra de treino
['lisboa', 'NUMERO', 'mai', 'reuter', 'indice', 'de', 'precos', 'dos', 'bens', 'nao', 'transaccionaveis', 'foi', 'em', 'abril', 'de', 'NUMERO', 'pct', 'contra', 'NUMERO', 'pct', 'no', 'mes', 'de', 'marco', 'anunciou', 'hoje', 'instituto', 'nacional', 'de', 'estatistica']


20 primeiras palavras da primeira amostra de teste
['lisboa', 'NUMERO', 'jun', 'reuter', 'indice', 'de', 'precos', 'no', 'consumidor', 'ipc', 'devera', 'registar', 'uma', 'evolucao', 'moderada', 'apesar', 'de', 'em', 'maio', 'ipc', 'homologo', 'ter', 'invertido', 'tendencia', 'descendente', 'dos', 'meses', 'anteriores', 'refere', 'sintese']


Neste momento, vamos treinar modelo próprio de embeddings baseado nos dados de treinamento. Para manipular as embeddings, iremos usar a biblioteca Gensim: https://radimrehurek.com/gensim/models/word2vec.html

In [88]:
sentencasEmbedding = dataset2_train
embeddingModel = Word2Vec(sentences = sentencasEmbedding,
                          vector_size = 200,
                          window = 3,
                          min_count = 1)

vocabSize = len(embeddingModel.wv)

print("\nTamanho do vocabulário do modelo: ", vocabSize)


Tamanho do vocabulário do modelo:  9507


In [89]:
def getDocvector(model, doc):
    """
    obtem o vetor de cada palavra de um documento e calcula um vetor medio
    """

    wordList = []
    for word in doc:

        try:
            vec = model.wv[word]
            wordList.append(vec)
        except:
            pass

    if len(wordList)>0:
        vetorMedio = np.mean( wordList, axis=0 )
    else:
        vetorMedio = np.zeros( model.wv.vector_size )

    return vetorMedio

In [90]:
def dataset2featureMatrix(dataset, embeddingModel):

    X_embedding = []
    for doc in dataset:
        vec = getDocvector(embeddingModel, doc)
        X_embedding.append(vec)

    X_embedding = np.array(X_embedding)
    return X_embedding


X_train_embedding = dataset2featureMatrix(dataset2_train, embeddingModel)
X_test_embedding = dataset2featureMatrix(dataset2_test, embeddingModel)

print(X_train_embedding.shape)
print(X_test_embedding.shape)


(2224, 200)
(557, 200)


_______
# Treinando um modelo de classificação por regressão logística


In [91]:
def classifica(X_train, X_test,
                Y_train, Y_test,
                formato = "TF"):

  model = skl.linear_model.LogisticRegression(
      C = 0.5, max_iter=500,
      random_state = 0
  )

  # normaliza os dados
  if formato=="embedding":
    scaler = skl.preprocessing.StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

  elif formato=="TF":
    scaler = skl.preprocessing.Normalizer(norm='l2')
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

  # treinando o método de classificação
  model.fit( X_train, Y_train )

  # classifica os dados de teste
  Y_pred = model.predict(X_test)

  print("\nAcurácia: ", skl.metrics.accuracy_score(Y_test, Y_pred))

In [92]:
print("Classificação Binário")
classifica(X_train_bin, X_test_bin, Y_train, Y_test,
            formato = "binario")

print("\nClassificação TF")
classifica(X_train_tf, X_test_tf, Y_train, Y_test,
            formato = "TF")

print("\nClassificação TF-IDF")
classifica(X_train_tfidf, X_test_tfidf, Y_train, Y_test,
            formato = "TF")

print("\nClassificação word embbeddings")
classifica(X_train_embedding, X_test_embedding, Y_train, Y_test,
            formato = "embedding")

Classificação Binário

Acurácia:  0.9569120287253142

Classificação TF

Acurácia:  0.9210053859964094

Classificação TF-IDF

Acurácia:  0.9425493716337523

Classificação word embbeddings

Acurácia:  0.9515260323159784


# Escolha dos Hiperparâmetros

Grid Search para definir o custo

In [93]:
def buscar_melhores_hiperparametros(X_train, Y_train):
    # Definindo o modelo com suporte a multiclasse
    model = LogisticRegression(
        max_iter=500,
        random_state=0,
        multi_class='multinomial',  # Alterado para 'multinomial' para suporte multiclasse
        solver='lbfgs'              # Solver compatível com 'multinomial'
    )

    # Definindo os valores dos hiperparâmetros para testar
    param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

    # Definindo o scorer AUC para multiclasse
    auc_scorer = make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr')

    # Inicializando a busca em grade com validação cruzada
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring=auc_scorer)

    # Treinando o modelo com a busca em grade
    grid_search.fit(X_train, Y_train)

    # Retorna o melhor modelo e o melhor valor de 'C' encontrado
    return grid_search.best_estimator_, grid_search.best_params_['C']


In [94]:
def classificar_com_melhores_hiperparametros(X_train, X_test, Y_train, Y_test, formato):
    # Normaliza os dados
    if formato == "embedding":
        scaler = skl_preprocessing.StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    elif formato == "TF":
        scaler = skl_preprocessing.Normalizer(norm='l2')
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)


    # Encontrando os melhores hiperparâmetros
    best_model, melhores_parametros = buscar_melhores_hiperparametros(X_train, Y_train)
    print(f"Melhores Hiperparâmetros: {melhores_parametros}")

    # Treinando o método de classificação com os melhores hiperparâmetros
    best_model.fit(X_train, Y_train)

    # Classifica os dados de teste
    Y_pred = best_model.predict(X_test)

    # Relatório de classificação
    print("\nRelatório de Classificação:\n", classification_report(Y_test, Y_pred))

    # Imprime a acurácia e AUC
    accuracy = skl_metrics.accuracy_score(Y_test, Y_pred)
    print(f"Acurácia: {accuracy:.4f}")

    y_proba = best_model.predict_proba(X_test)
    auc = roc_auc_score(Y_test, y_proba, multi_class='ovo')
    print(f"AUC: {auc:.4f}")

#### Testando TF

In [95]:
best_model_tf_grid = classificar_com_melhores_hiperparametros(X_train_tf, X_test_tf, Y_train, Y_test, 'TF')

Melhores Hiperparâmetros: 100

Relatório de Classificação:
                 precision    recall  f1-score   support

CorpIndustrial       0.50      0.29      0.36         7
      Economia       0.90      0.95      0.93       118
     GovSocial       0.85      0.61      0.71        18
      Mercados       0.99      0.99      0.99       414

      accuracy                           0.96       557
     macro avg       0.81      0.71      0.75       557
  weighted avg       0.96      0.96      0.96       557

Acurácia: 0.9605
AUC: 0.9493


#### Testando TF-IDF

In [96]:
best_model_tfidf_grid = classificar_com_melhores_hiperparametros(X_train_tfidf, X_test_tfidf, Y_train, Y_test, 'TF')

Melhores Hiperparâmetros: 100

Relatório de Classificação:
                 precision    recall  f1-score   support

CorpIndustrial       0.67      0.29      0.40         7
      Economia       0.90      0.95      0.92       118
     GovSocial       0.86      0.67      0.75        18
      Mercados       0.99      0.99      0.99       414

      accuracy                           0.96       557
     macro avg       0.85      0.72      0.76       557
  weighted avg       0.96      0.96      0.96       557

Acurácia: 0.9605
AUC: 0.9561


#### Testando Binário

In [97]:
best_model_tfbin_grid = classificar_com_melhores_hiperparametros(X_train_bin, X_test_bin, Y_train, Y_test, 'binario')

Melhores Hiperparâmetros: 1

Relatório de Classificação:
                 precision    recall  f1-score   support

CorpIndustrial       0.50      0.29      0.36         7
      Economia       0.90      0.96      0.93       118
     GovSocial       0.73      0.44      0.55        18
      Mercados       0.98      0.99      0.99       414

      accuracy                           0.96       557
     macro avg       0.78      0.67      0.71       557
  weighted avg       0.95      0.96      0.95       557

Acurácia: 0.9551
AUC: 0.9525


#### Testando Word Embeddings

In [98]:
best_model_embedding_grid = classificar_com_melhores_hiperparametros(X_train_embedding, X_test_embedding, Y_train, Y_test, 'embedding')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Melhores Hiperparâmetros: 0.1

Relatório de Classificação:
                 precision    recall  f1-score   support

CorpIndustrial       0.60      0.43      0.50         7
      Economia       0.88      0.97      0.92       118
     GovSocial       1.00      0.50      0.67        18
      Mercados       0.99      0.99      0.99       414

      accuracy                           0.96       557
     macro avg       0.87      0.72      0.77       557
  weighted avg       0.96      0.96      0.96       557

Acurácia: 0.9587
AUC: 0.9064


---
# Previsão do conjunto de teste

In [67]:
# Função para preparar os dados de teste
def prepare_test_data(df_test):
    # Aplicar a função de extração de textos do XML no DataFrame de teste
    df_test = apply_extraction(df_test)

    # Re-organizar as colunas do DataFrame
    df_test = df_test[['ID', 'TITULO', 'TEXTO']]

    # Aplicar a função de pré-processamento no texto das amostras de teste
    df_test['TEXTO'] = df_test['TEXTO'].apply(preprocessing_portuguese)

    return df_test

In [104]:
def classificar_e_treinar_com_melhores_hiperparametros(X_train, X_val, y_train, y_val):
    # Encontrando os melhores hiperparâmetros
    best_model, melhores_parametros = buscar_melhores_hiperparametros(X_train, y_train)
    print(f"Melhores Hiperparâmetros: {melhores_parametros}")

    scaler = skl_preprocessing.Normalizer(norm='l2')
    X_val = scaler.fit_transform(X_val)


    # Fazendo previsões no conjunto de validação
    y_pred = best_model.predict(X_val)

    # Avaliando a acurácia
    accuracy = skl_metrics.accuracy_score(y_val, y_pred)
    print(f"Acurácia: {accuracy:.4f}")

    # Relatório de classificação
    print("\nRelatório de Classificação:\n", skl_metrics.classification_report(y_val, y_pred,zero_division=0))

    # Matriz de confusão
    print("\nMatriz de Confusão:\n", skl_metrics.confusion_matrix(y_val, y_pred))

    # Calculando a AUC
    y_proba = best_model.predict_proba(X_val)
    auc = roc_auc_score(y_val, y_proba, multi_class='ovo')
    print(f"AUC: {auc:.4f}")

    return best_model


In [115]:
# Função para obter as probabilidades e gerar o arquivo de submissão
def generate_submission_file(classifier, X_test, df_test, filename='submission.csv'):
    # Obter as probabilidades das classes
    y_proba = classifier.predict_proba(X_test)

    # Criar um DataFrame com as probabilidades
    submission_df = pd.DataFrame(y_proba, columns=le.classes_)

    # Adicionar a coluna ID
    submission_df.insert(0, 'ID', df_test['ID'])

    # Renomear as colunas para o formato exigido
    submission_df.columns = ['ID', 'CorpIndustrial', 'Economia', 'GovSocial', 'Mercados']

    # Salvar o DataFrame como um arquivo CSV
    submission_df.to_csv(filename, index=False, float_format='%.5f')
    print(f"Arquivo de submissão salvo como {filename}")

In [70]:
# Carregar e preparar os dados de teste
df_test = prepare_test_data(df_teste)

In [71]:
# Como o TF-IDF teve a maior AUC, então será aplicado apenas esse metodo
X_test_tf = vectorizer.transform(df_test['TEXTO'])
X_test_tfidf = tfidf_model.transform(X_test_tf)

In [105]:
# Divisão dos dados de treino em treino e validação
X_train_part, X_val_part, y_train_part, y_val_part = train_test_split(X_train_tfidf, Y_train, test_size=0.2, random_state=42, stratify=Y_train)

# Treinar e avaliar o modelo com busca aleatória para TF-IDF
print("\n\nTreinando com o formato TF-IDF usando busca por grid")
print(X_train_part.shape, X_val_part.shape, y_train_part.shape)
best_model_tfidf = classificar_e_treinar_com_melhores_hiperparametros(X_train_part, X_val_part, y_train_part, y_val_part)



Treinando com o formato TF-IDF usando busca por grid
(1779, 9506) (445, 9506) (1779,)
Melhores Hiperparâmetros: 100
Acurácia: 0.9843

Relatório de Classificação:
                 precision    recall  f1-score   support

CorpIndustrial       1.00      0.50      0.67         6
      Economia       0.97      0.98      0.97        94
     GovSocial       0.87      0.93      0.90        14
      Mercados       0.99      1.00      1.00       331

      accuracy                           0.98       445
     macro avg       0.96      0.85      0.88       445
  weighted avg       0.98      0.98      0.98       445


Matriz de Confusão:
 [[  3   1   1   1]
 [  0  92   1   1]
 [  0   1  13   0]
 [  0   1   0 330]]
AUC: 0.9882


In [116]:
# Gerar o arquivo de submissão usando o modelo treinado
generate_submission_file(best_model_tfidf, X_test_tfidf, df_test, filename='submission_files/regressao/submission_tfidf.csv')

Arquivo de submissão salvo como submission_files/regressao/submission_tfidf.csv
